In [35]:
import pandas as pd
# from utils import select_min_pay, read_input_file, export_file
import glob
import numpy as np
import pandas as pd
import glob
import logging
import win32com.client as win32
import os
import jaydebeapi
from const import QUERY, CONNECT
import oracledb
import getpass
import json

In [36]:
file_current = glob.glob("./input/acc_current/*") 
file_current = file_current[0].split('\\')[1]
file_current

'Acc_20250201_0830.csv'

In [37]:
from utils  import read_file,clean_column_names

In [38]:
data_digit1 = read_file('./input/acc_current/',file_current)


In [39]:
data_digit1 = clean_column_names(data_digit1)


In [40]:
data_digit1

,basedate,contract_no,product_code,overdue_start_date,overduedays_morning,overduecnt_morning,overduedays,overduecnt,loan_amt,loan_balance,...,overduecrusage,overdueoutputtax,overduecollectionfee,overduevat,overduestamptaxamt,overduencbtaxamt,overdueetcfeeamt,lawcost,lastpaymentdt,lastpaymentamt
0,20250201,9010221122000949,A501:TL,2024-09-01,153,5:OD4,153,5:OD4,74164.44,38086.28,...,941.16,0.0,467.3,32.7,0,0,0.0,0.0,2025-01-30,3000.00
1,20250201,9010221122000931,A501:TL,,0,99:NORMAL,0,99:NORMAL,41659.76,13950.38,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2024-09-27,20971.92
2,20250201,9010221122000922,A501:TL,2023-08-31,0,98:Write-Off,0,98:Write-Off,34238.99,23851.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-01-31,1300.00
3,20250201,9010221122000914,A501:TL,2024-02-29,0,98:Write-Off,0,98:Write-Off,16118.33,13559.55,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2023-12-27,550.00
4,20250201,9010221122000906,A501:TL,2022-11-30,0,98:Write-Off,0,98:Write-Off,35235.40,31005.17,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2022-09-30,2100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366988,20250201,1000158090014831,A401:ML,2017-02-01,0,99:NORMAL,0,99:NORMAL,25500.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-01-31,1580.00
1366989,20250201,1000158090014814,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,24600.00,30.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-31,1260.00
1366990,20250201,1000158090008580,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-07-16,4000.00
1366991,20250201,1000158090005394,A401:ML,2017-08-01,0,99:NORMAL,0,99:NORMAL,25900.00,100.00,...,0.00,0.0,0.0,0.0,0,0,0.0,0.0,2017-08-01,2600.00


history weight

In [41]:
conn = jaydebeapi.connect(
        CONNECT.DB,
        CONNECT.PORT,
        CONNECT.USER,
        CONNECT.CNN,
        )
cur = conn.cursor()  

query = pd.read_sql(f'''
      SELECT * FROM SIREETRON.bucket_score  
    ''', conn) 
conn.close()

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_54608\996401911.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql(f'''


In [42]:
query

,ID,BUCKET,SCORE_INDEX,BUCKET_SCORE
0,1,99:NORMAL,1,0
1,2,1:XD,1,1
2,3,2:OD1,2,2
3,4,3:OD2,2,3
4,5,4:OD3,3,4
5,6,5:OD4,3,5
6,7,6:OD5,3,6
7,8,7:OD6,3,7
8,9,8:OD7,3,8
9,10,9:OD8,3,9


In [43]:

def all_history(date,item) :
    data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')
    print(f'./input/history/{date}')
    data_mnt['Loan No'] = data_mnt['Loan No'].astype(str)
    data_mnt = data_mnt[['Loan No','OverdueCnt_Morning']].merge(query, left_on='OverdueCnt_Morning', right_on='BUCKET', how='left')
    data_mnt = data_mnt.rename(columns={'BUCKET_SCORE': f'BUCKET_SCORE{item}mnt'})
    # data_mnt = data_mnt.rename(columns={'OverdueCnt_Morning': f'OverdueCnt_Morning{item}'})
    return data_mnt[['Loan No',f'BUCKET_SCORE{item}mnt']]



In [45]:
data_target = pd.read_excel(f'./input/acc_current/02_Feb_Template Performance Overall as of 28-02-2025_FINAL.xlsx',sheet_name='DATA')
data_target

,Base date,Customer No,Customer ID No,Principal balance,Total Unclaimed Amt.,Repayment Target Amount,Overdue days(Morning),Overdue Cnt(Morning),Overdue days,Num of Overdue months,...,EVER,Due,OD TERM,OD Day_Today,OD,Status,Payment,Last Payment Date,Product,Status LM
0,02-02-2568,12160022732,3100400473465,86981.96,0.0,2565.94,1,1,1,1,...,Y,1,NaN,0,99:NORMAL,SAVE,1582.00,2025-02-18,RL ML TL,SAVE
1,02-02-2568,12170054801,3471200908156,67545.86,0.0,2071.19,1,1,1,1,...,Y,1,NaN,0,99:NORMAL,SAVE,2075.00,2025-02-07,RL ML TL,SAVE
2,02-02-2568,12170040400,1849800083805,56484.46,0.0,1363.61,1,1,1,1,...,Y,1,NaN,0,99:NORMAL,SAVE,1500.00,2025-02-05,RL ML TL,SAVE
3,02-02-2568,12180045507,3361300226376,37956.52,0.0,1188.38,1,1,1,1,...,Y,1,NaN,0,99:NORMAL,SAVE,1188.69,2025-02-05,RL ML TL,SAVE
4,02-02-2568,12170018129,1330200006075,31166.48,0.0,876.09,1,1,1,1,...,Y,1,NaN,0,99:NORMAL,SAVE,700.00,2025-02-13,RL ML TL,SAVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,18-02-2568,14220002608,1450900019144,11881.36,0.0,5734.00,154,6,154,6,...,N,17,NaN,165,6:OD5,UNSAVE,0.00,NaT,RL ML TL,UNSAVE
36052,18-02-2568,13210013404,5200100011313,223915.63,0.0,80538.00,185,7,185,7,...,N,17,NaN,0,98:Write-Off,UNSAVE,0.00,NaT,RL ML TL,UNSAVE
36053,18-02-2568,13210012399,3560600128873,84210.89,0.0,55636.00,185,7,185,7,...,N,17,NaN,0,98:Write-Off,UNSAVE,0.00,NaT,RL ML TL,UNSAVE
36054,18-02-2568,13200001071,3100903395243,45130.78,0.0,24910.00,185,7,185,7,...,N,17,NaN,0,98:Write-Off,UNSAVE,0.00,NaT,RL ML TL,UNSAVE


In [46]:
data_target  = data_target.rename(columns={'Loan No.': 'Loan No'})

In [7]:
data_target[['Loan No']]

,Loan No
0,8002059101048961
1,5000160091046498
2,5000460051052371
3,5000361021074351
4,5000160071046565
...,...
36051,7220150100000228
36052,9000221102011468
36053,9000220082000911
36054,9000220102001658


In [47]:
data_target['Loan No'] = data_target['Loan No'].astype(str)
data_digit1 = data_target[['Loan No']]

In [48]:
import re

In [49]:
files = glob.glob("./input/acc_history_monthly/*.csv") 

In [50]:
# files = []
processed_files = []  # Store modified file names

for i in files:
    file = i.split('\\')[-1] 
    print(file)
    processed_files.append(file)  
print(processed_files)

Acc_20240901_0830.csv
Acc_20241001_0830.csv
Acc_20241101_0830.csv
Acc_20241201_0830.csv
Acc_20250101_0830.csv
Acc_20250201_0830.csv
['Acc_20240901_0830.csv', 'Acc_20241001_0830.csv', 'Acc_20241101_0830.csv', 'Acc_20241201_0830.csv', 'Acc_20250101_0830.csv', 'Acc_20250201_0830.csv']


In [51]:
def transform_files(files):
    files.sort(reverse=True)
    latest_files = files
    return [[file,  re.sub(r'\.csv$', '', file)] for file in latest_files]

In [52]:
his = transform_files(processed_files)

In [53]:
his

[['Acc_20250201_0830.csv', 'Acc_20250201_0830'],
 ['Acc_20250101_0830.csv', 'Acc_20250101_0830'],
 ['Acc_20241201_0830.csv', 'Acc_20241201_0830'],
 ['Acc_20241101_0830.csv', 'Acc_20241101_0830'],
 ['Acc_20241001_0830.csv', 'Acc_20241001_0830'],
 ['Acc_20240901_0830.csv', 'Acc_20240901_0830']]

In [54]:
data_digit1

,Loan No
0,8002059101048961
1,5000160091046498
2,5000460051052371
3,5000361021074351
4,5000160071046565
...,...
36051,7220150100000228
36052,9000221102011468
36053,9000220082000911
36054,9000220102001658


In [55]:
# his = [ ['20250301', '0'],['20250201', '1'],['20250101','2'],['20241201','3'],['20241101','4'],['20241001','5']]
for i in his[0:]:
    print(f"Processing date: {i[0]}")
    print((i[0], i[1]))
    data = all_history(i[0], i[1])
    data_digit1 = data_digit1.merge(data, on='Loan No', how='outer')

Processing date: Acc_20250201_0830.csv
('Acc_20250201_0830.csv', 'Acc_20250201_0830')
./input/history/Acc_20250201_0830.csv
Processing date: Acc_20250101_0830.csv
('Acc_20250101_0830.csv', 'Acc_20250101_0830')
./input/history/Acc_20250101_0830.csv
Processing date: Acc_20241201_0830.csv
('Acc_20241201_0830.csv', 'Acc_20241201_0830')
./input/history/Acc_20241201_0830.csv
Processing date: Acc_20241101_0830.csv
('Acc_20241101_0830.csv', 'Acc_20241101_0830')


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_54608\2874079498.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')


./input/history/Acc_20241101_0830.csv
Processing date: Acc_20241001_0830.csv
('Acc_20241001_0830.csv', 'Acc_20241001_0830')


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_54608\2874079498.py:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')


./input/history/Acc_20241001_0830.csv
Processing date: Acc_20240901_0830.csv
('Acc_20240901_0830.csv', 'Acc_20240901_0830')


C:\Users\Sireetron\AppData\Local\Temp\ipykernel_54608\2874079498.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data_mnt = pd.read_csv(f'./input/acc_history_monthly/{date}')


./input/history/Acc_20240901_0830.csv


In [56]:
data_digit1

,Loan No,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt
0,1000158080014811,0,0,0,0,0,NaN
1,1000158090005394,0,0,0,0,0,NaN
2,1000158090008580,0,0,0,0,0,NaN
3,1000158090014814,0,0,0,0,0,NaN
4,1000158090014831,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...
1569387,9010221122000914,13,13,13,13,13,13
1569388,9010221122000922,13,13,13,13,13,13
1569389,9010221122000931,0,0,0,0,0,0
1569390,9010221122000949,5,5,5,5,5,5


In [57]:
data_digit1['AVG_DIGIT1'] = data_digit1.loc[:, data_digit1.columns.str.startswith('BUCKET_SCORE')].apply(pd.to_numeric, errors='coerce').sum(axis=1).fillna(0).astype(int)/6

In [58]:
data_digit1

,Loan No,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,NaN,0.0
1,1000158090005394,0,0,0,0,0,NaN,0.0
2,1000158090008580,0,0,0,0,0,NaN,0.0
3,1000158090014814,0,0,0,0,0,NaN,0.0
4,1000158090014831,0,0,0,0,0,NaN,0.0
...,...,...,...,...,...,...,...,...
1569387,9010221122000914,13,13,13,13,13,13,13.0
1569388,9010221122000922,13,13,13,13,13,13,13.0
1569389,9010221122000931,0,0,0,0,0,0,0.0
1569390,9010221122000949,5,5,5,5,5,5,5.0


In [ ]:
# conditions = [
#     (data_digit1['AVG_DIGIT1_Fhn'] > 3),
#     (data_digit1['AVG_DIGIT1_Fhn'] > 1),
#     (data_digit1['AVG_DIGIT1_Fhn'] == 0)
# ]

# # Define corresponding values
# choices = ['H', 'M', 'L']

# # Apply the mapping logic
# data_digit1['AVG_DIGIT1_digit_Fhn'] = np.select(conditions, choices, default='M')

In [283]:
# data_digit1[]

,Loan No,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt
0,1000158080014811,0,0,0,0,0,0
1,1000158090005394,0,0,0,0,0,0
2,1000158090008580,0,0,0,0,0,0
3,1000158090014814,0,0,0,0,0,0
4,1000158090014831,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1633098,9010221122000914,13,13,13,13,13,13
1633099,9010221122000922,13,13,13,13,13,13
1633100,9010221122000931,0,0,0,0,0,0
1633101,9010221122000949,5,5,5,5,5,5


In [10]:
data_digit1

,Loan No,BUCKET_SCORE0mnt,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,0,0.0
1,1000158090005394,0,0,0,0,0,0,0.0
2,1000158090008580,0,0,0,0,0,0,0.0
3,1000158090014814,0,0,0,0,0,0,0.0
4,1000158090014831,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...
1633098,9010221122000914,13,13,13,13,13,13,13.0
1633099,9010221122000922,13,13,13,13,13,13,13.0
1633100,9010221122000931,0,0,0,0,0,0,0.0
1633101,9010221122000949,5,5,5,5,5,5,5.0


In [59]:
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)

In [60]:
data_digit1

,Loan No,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,NaN,0.0
1,1000158090005394,0,0,0,0,0,NaN,0.0
2,1000158090008580,0,0,0,0,0,NaN,0.0
3,1000158090014814,0,0,0,0,0,NaN,0.0
4,1000158090014831,0,0,0,0,0,NaN,0.0
...,...,...,...,...,...,...,...,...
1569387,9010221122000914,13,13,13,13,13,13,13.0
1569388,9010221122000922,13,13,13,13,13,13,13.0
1569389,9010221122000931,0,0,0,0,0,0,0.0
1569390,9010221122000949,5,5,5,5,5,5,5.0


datatarget

In [61]:
data_digit1

,Loan No,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,NaN,0.0
1,1000158090005394,0,0,0,0,0,NaN,0.0
2,1000158090008580,0,0,0,0,0,NaN,0.0
3,1000158090014814,0,0,0,0,0,NaN,0.0
4,1000158090014831,0,0,0,0,0,NaN,0.0
...,...,...,...,...,...,...,...,...
1569387,9010221122000914,13,13,13,13,13,13,13.0
1569388,9010221122000922,13,13,13,13,13,13,13.0
1569389,9010221122000931,0,0,0,0,0,0,0.0
1569390,9010221122000949,5,5,5,5,5,5,5.0


In [62]:
data_digit1

,Loan No,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,1000158080014811,0,0,0,0,0,NaN,0.0
1,1000158090005394,0,0,0,0,0,NaN,0.0
2,1000158090008580,0,0,0,0,0,NaN,0.0
3,1000158090014814,0,0,0,0,0,NaN,0.0
4,1000158090014831,0,0,0,0,0,NaN,0.0
...,...,...,...,...,...,...,...,...
1569387,9010221122000914,13,13,13,13,13,13,13.0
1569388,9010221122000922,13,13,13,13,13,13,13.0
1569389,9010221122000931,0,0,0,0,0,0,0.0
1569390,9010221122000949,5,5,5,5,5,5,5.0


In [63]:
data_digit1 = data_target.merge(data_digit1,left_on='Loan No',right_on='Loan No',how='left')

In [64]:
data_digit1

,Base date,Customer No,Customer ID No,Principal balance,Total Unclaimed Amt.,Repayment Target Amount,Overdue days(Morning),Overdue Cnt(Morning),Overdue days,Num of Overdue months,...,Last Payment Date,Product,Status LM,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,02-02-2568,12160022732,3100400473465,86981.96,0.0,2565.94,1,1,1,1,...,2025-02-18,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
1,02-02-2568,12170054801,3471200908156,67545.86,0.0,2071.19,1,1,1,1,...,2025-02-07,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
2,02-02-2568,12170040400,1849800083805,56484.46,0.0,1363.61,1,1,1,1,...,2025-02-05,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
3,02-02-2568,12180045507,3361300226376,37956.52,0.0,1188.38,1,1,1,1,...,2025-02-05,RL ML TL,SAVE,0,0,0,1,0,NaN,0.166667
4,02-02-2568,12170018129,1330200006075,31166.48,0.0,876.09,1,1,1,1,...,2025-02-13,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,18-02-2568,14220002608,1450900019144,11881.36,0.0,5734.00,154,6,154,6,...,NaT,RL ML TL,UNSAVE,5,4,3,2,1,NaN,2.500000
36052,18-02-2568,13210013404,5200100011313,223915.63,0.0,80538.00,185,7,185,7,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,2,3.666667
36053,18-02-2568,13210012399,3560600128873,84210.89,0.0,55636.00,185,7,185,7,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,1,3.500000
36054,18-02-2568,13200001071,3100903395243,45130.78,0.0,24910.00,185,7,185,7,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,3,3.833333


Perfomance

In [66]:
data_performance = pd.read_csv('./input/performance_peronal_info/performance.csv')
# data_AC = pd.read_csv('./input/Acc_20250307_0830.csv')

C:\Users\Sireetron\AppData\Local\Temp\ipykernel_54608\3401514148.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data_performance = pd.read_csv('./input/performance_peronal_info/performance.csv')


In [67]:
data_performance

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,MOB,DIGIT2INCOME,DIGIT2AGE,DIGIT2JOB,TOTAL_DIGIT2
0,2025-03-19 00:00:00.000,5000161051018231,1110300202706,9750.0,9.0,PRODUCTION LINE LABOR,2018-05-07 00:00:00.000,82.0,27.0,0.00,7,3,3,3,3
1,2025-03-19 00:00:00.000,1000161051042121,1110300202706,9750.0,9.0,PRODUCTION LINE LABOR,2018-05-14 00:00:00.000,82.0,27.0,801.00,7,3,3,3,3
2,2025-03-19 00:00:00.000,5002461061050530,1110300203281,10140.0,5.0,OFFICE WORK,2018-06-13 00:00:00.000,81.0,27.0,0.00,7,3,3,1,3
3,2025-03-19 00:00:00.000,5000361021006968,1110300205578,9300.0,10.0,GENERAL LABOR,2018-02-02 00:00:00.000,85.0,27.0,13890.67,7,3,3,2,3
4,2025-03-19 00:00:00.000,5000461041075891,1110300208135,13737.0,9.0,PRODUCTION LINE LABOR,2018-04-26 00:00:00.000,82.0,27.0,0.00,7,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5386968,2025-03-19 00:00:00.000,7250170104680761,3800400714270,12500.0,4.0,SPECIALIZATION,2025-01-31 00:00:00.000,1.0,41.0,0.00,6,3,2,1,2
5386969,2025-03-19 00:00:00.000,7241270104483495,1301400032895,17500.0,5.0,OFFICE WORK,2024-12-28 00:00:00.000,2.0,39.0,0.00,6,3,3,1,3
5386970,2025-03-19 00:00:00.000,7250170104615532,5361200029818,50500.0,2.0,UNREGISTERED BUSINESS OWNER,2025-01-19 00:00:00.000,2.0,59.0,24211.25,6,2,1,3,2
5386971,2025-03-19 00:00:00.000,7250170104600813,3720200607489,17500.0,9.0,PRODUCTION LINE LABOR,2025-01-16 00:00:00.000,2.0,46.0,0.00,6,3,2,3,3


In [68]:
data_digit1['Loan No'] = data_digit1['Loan No'].astype(str)
data_performance['CONTRACT_NO']= data_performance['CONTRACT_NO'].astype(str)

In [ ]:
# test = data_performance.merge(data_digit1, left_on='CONTRACT_NO', right_on='Loan No', how='right')

In [69]:
data_all_digit =  data_performance.merge(data_digit1, left_on='CONTRACT_NO', right_on='Loan No', how='right')


In [70]:
data_digit1

,Base date,Customer No,Customer ID No,Principal balance,Total Unclaimed Amt.,Repayment Target Amount,Overdue days(Morning),Overdue Cnt(Morning),Overdue days,Num of Overdue months,...,Last Payment Date,Product,Status LM,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,02-02-2568,12160022732,3100400473465,86981.96,0.0,2565.94,1,1,1,1,...,2025-02-18,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
1,02-02-2568,12170054801,3471200908156,67545.86,0.0,2071.19,1,1,1,1,...,2025-02-07,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
2,02-02-2568,12170040400,1849800083805,56484.46,0.0,1363.61,1,1,1,1,...,2025-02-05,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
3,02-02-2568,12180045507,3361300226376,37956.52,0.0,1188.38,1,1,1,1,...,2025-02-05,RL ML TL,SAVE,0,0,0,1,0,NaN,0.166667
4,02-02-2568,12170018129,1330200006075,31166.48,0.0,876.09,1,1,1,1,...,2025-02-13,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,18-02-2568,14220002608,1450900019144,11881.36,0.0,5734.00,154,6,154,6,...,NaT,RL ML TL,UNSAVE,5,4,3,2,1,NaN,2.500000
36052,18-02-2568,13210013404,5200100011313,223915.63,0.0,80538.00,185,7,185,7,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,2,3.666667
36053,18-02-2568,13210012399,3560600128873,84210.89,0.0,55636.00,185,7,185,7,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,1,3.500000
36054,18-02-2568,13200001071,3100903395243,45130.78,0.0,24910.00,185,7,185,7,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,3,3.833333


In [108]:
data_all_digit

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,Last Payment Date,Product,Status LM,BUCKET_SCOREAcc_20250301_0830mnt,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,AVG_DIGIT1
0,2025-03-19 00:00:00.000,8002059101048961,3100400473465,19390.0,8.0,SERVICE JOB,2016-10-13 00:00:00.000,101.0,56.0,86981.96,...,2025-02-18,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
1,2025-03-19 00:00:00.000,5000160091046498,3471200908156,30000.0,10.0,GENERAL LABOR,2017-09-22 00:00:00.000,89.0,53.0,66377.64,...,2025-02-07,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
2,2025-03-19 00:00:00.000,5000460051052371,1849800083805,20500.0,9.0,PRODUCTION LINE LABOR,2017-05-30 00:00:00.000,93.0,33.0,55554.07,...,2025-02-05,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
3,2025-03-19 00:00:00.000,5000361021074351,3361300226376,45180.0,9.0,PRODUCTION LINE LABOR,2018-02-22 00:00:00.000,84.0,45.0,37242.57,...,2025-02-05,RL ML TL,SAVE,0,0,0,0,1,0,0.166667
4,2025-03-19 00:00:00.000,5000160071046565,1330200006075,15500.0,5.0,OFFICE WORK,2017-07-21 00:00:00.000,91.0,40.0,31031.06,...,2025-02-13,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,2025-03-19 00:00:00.000,7220150100000228,1450900019144,15387.0,7.0,DRIVING,2022-01-26 00:00:00.000,37.0,39.0,11881.36,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,1,3.500000
36052,2025-03-19 00:00:00.000,9000221102011468,5200100011313,113934.0,5.0,OFFICE WORK,2021-10-27 00:00:00.000,40.0,54.0,223915.63,...,NaT,RL ML TL,UNSAVE,13,6,5,4,3,2,5.500000
36053,2025-03-19 00:00:00.000,9000220082000911,3560600128873,130317.0,3.0,SENIOR MANAGEMENT,2020-08-13 00:00:00.000,55.0,48.0,82910.89,...,NaT,RL ML TL,UNSAVE,13,6,5,4,3,2,5.500000
36054,2025-03-19 00:00:00.000,9000220102001658,3100903395243,64750.0,3.0,SENIOR MANAGEMENT,2020-10-19 00:00:00.000,53.0,49.0,42260.78,...,NaT,RL ML TL,UNSAVE,13,6,5,4,3,2,5.500000


In [71]:
data_all_digit.groupby('TOTAL_DIGIT2').size()

TOTAL_DIGIT2
1     1767
2     8512
3    25777
dtype: int64

In [72]:
data_all_digit['MOB'] = pd.to_numeric(data_all_digit['MOB'], errors='coerce').fillna(0).astype(int)

In [73]:
data_all_digit['MOB']

0        7
1        7
2        7
3        7
4        7
        ..
36051    7
36052    7
36053    7
36054    7
36055    7
Name: MOB, Length: 36056, dtype: int64

In [74]:
data_all_digit

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,Last Payment Date,Product,Status LM,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1
0,2025-03-19 00:00:00.000,8002059101048961,3100400473465,19390.0,8.0,SERVICE JOB,2016-10-13 00:00:00.000,101.0,56.0,86981.96,...,2025-02-18,RL ML TL,SAVE,0,0,0,0,0,0,0.000000
1,2025-03-19 00:00:00.000,5000160091046498,3471200908156,30000.0,10.0,GENERAL LABOR,2017-09-22 00:00:00.000,89.0,53.0,66377.64,...,2025-02-07,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
2,2025-03-19 00:00:00.000,5000460051052371,1849800083805,20500.0,9.0,PRODUCTION LINE LABOR,2017-05-30 00:00:00.000,93.0,33.0,55554.07,...,2025-02-05,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
3,2025-03-19 00:00:00.000,5000361021074351,3361300226376,45180.0,9.0,PRODUCTION LINE LABOR,2018-02-22 00:00:00.000,84.0,45.0,37242.57,...,2025-02-05,RL ML TL,SAVE,0,0,0,1,0,NaN,0.166667
4,2025-03-19 00:00:00.000,5000160071046565,1330200006075,15500.0,5.0,OFFICE WORK,2017-07-21 00:00:00.000,91.0,40.0,31031.06,...,2025-02-13,RL ML TL,SAVE,0,0,0,0,0,NaN,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,2025-03-19 00:00:00.000,7220150100000228,1450900019144,15387.0,7.0,DRIVING,2022-01-26 00:00:00.000,37.0,39.0,11881.36,...,NaT,RL ML TL,UNSAVE,5,4,3,2,1,NaN,2.500000
36052,2025-03-19 00:00:00.000,9000221102011468,5200100011313,113934.0,5.0,OFFICE WORK,2021-10-27 00:00:00.000,40.0,54.0,223915.63,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,2,3.666667
36053,2025-03-19 00:00:00.000,9000220082000911,3560600128873,130317.0,3.0,SENIOR MANAGEMENT,2020-08-13 00:00:00.000,55.0,48.0,82910.89,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,1,3.500000
36054,2025-03-19 00:00:00.000,9000220102001658,3100903395243,64750.0,3.0,SENIOR MANAGEMENT,2020-10-19 00:00:00.000,53.0,49.0,42260.78,...,NaT,RL ML TL,UNSAVE,6,5,4,3,2,3,3.833333


In [75]:
data_all_digit.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'NATIONAL_ID', 'GROSS_INCOME',
       'OCCUPATION_TYPE', 'OCCUPATION_NAME', 'CONTRACT_DATE',
       'CONTRACT_PERIOD', 'AGE', 'PRINCIPAL_BAL',
       ...
       'Last Payment Date', 'Product', 'Status LM',
       'BUCKET_SCOREAcc_20250201_0830mnt', 'BUCKET_SCOREAcc_20250101_0830mnt',
       'BUCKET_SCOREAcc_20241201_0830mnt', 'BUCKET_SCOREAcc_20241101_0830mnt',
       'BUCKET_SCOREAcc_20241001_0830mnt', 'BUCKET_SCOREAcc_20240901_0830mnt',
       'AVG_DIGIT1'],
      dtype='object', length=112)

TDH

In [79]:
files = glob.glob("./Input/ar_all/*.*")
print(files)
    # logger.info('Start Processing...')
    # logger.info('Reading input files...')


['./Input/ar_all\\TDR_ALL_LIST_FEB_2025.xlsx']


In [80]:
df_list = []  # Store individual DataFrames

for file in files:
    print(f"Reading {file}...")
    path = glob.glob(f"{file}")[0].split('\\')[-1]
    # print(path)
    df = read_file('./input/ar_all/',path)
    df_list.append(df)

Reading ./Input/ar_all\TDR_ALL_LIST_FEB_2025.xlsx...


In [81]:
df_list

[      H     Customer No           Loan No Product Project Code  \
 0     D  ="11150019114"  1000158110010231      TL         P012   
 1     D  ="11150011505"  1000158120003165      TL         P014   
 2     D  ="11160019055"  1000159040033450      TL         P018   
 3     D  ="11160011523"  1000159050040751      TL         P014   
 4     D  ="11160048826"  1000159050046961      TL         P018   
 ...  ..             ...               ...     ...          ...   
 4385  D  ="13210012277"  9010221122000507      TL   PLD2021070   
 4386  D  ="13210012644"  9010221122000566      TL   PLD2021035   
 4387  D  ="13210001266"  9010221122000817      TL   PLD2021035   
 4388  D  ="13210005782"  9010221122000931      TL   PLD2020023   
 4389  D  ="13210002840"  9010221122000949      TL   PLD2020023   
 
                              Project Code Name Promotion Code Promotion Name  \
 0                                  Activity PL                                 
 1                              

In [82]:
data_tdr = pd.concat(df_list, ignore_index=True)
data_tdr

,H,Customer No,Loan No,Product,Project Code,Project Code Name,Promotion Code,Promotion Name,Loan Date,Maturity Date,...,AS Output Grade Code,Job Type Code,Less Than 1 Year Balance,Less Than 3 Year Balance,Less Than 5 Year Balance,Exceeding 5 Year Balance,RL Suspention Y/N,Customer Total Credit Line,SAM Y/N,SUING Y/N
0,D,"=""11150019114""",1000158110010231,TL,P012,Activity PL,,,2015-11-12,2017-12-01,...,,6,100.00,0.00,0.0,0.0,N,31500.00,N,N
1,D,"=""11150011505""",1000158120003165,TL,P014,Walk in PL,,,2015-12-11,2018-07-01,...,,5,100.00,0.00,0.0,0.0,N,30600.00,N,N
2,D,"=""11160019055""",1000159040033450,TL,P018,Webpak,,,2016-04-27,2018-05-01,...,,9,90.00,0.00,0.0,0.0,N,32500.00,N,N
3,D,"=""11160011523""",1000159050040751,TL,P014,Walk in PL,,,2016-05-26,2019-06-01,...,,8,1.60,0.00,0.0,0.0,N,38100.00,N,N
4,D,"=""11160048826""",1000159050046961,TL,P018,Webpak,,,2016-05-30,2019-06-01,...,,9,90.00,0.00,0.0,0.0,N,31500.00,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4385,D,"=""13210012277""",9010221122000507,TL,PLD2021070,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,14972.66,40931.85,409.2,0.0,N,205411.79,N,N
4386,D,"=""13210012644""",9010221122000566,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2026-01-01,...,,,7434.04,274.10,0.0,0.0,N,45667.25,N,N
4387,D,"=""13210001266""",9010221122000817,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,7546.63,20065.63,0.0,0.0,N,89316.84,N,N
4388,D,"=""13210005782""",9010221122000931,TL,PLD2020023,ปรับโครงสร้างหนี้_จากRL_22%_48ด,,,2021-12-08,2025-12-01,...,,,16696.03,0.00,0.0,0.0,N,106159.76,N,N


In [34]:
clean_column_names(data_tdr)

,h,customer_no,contract_no,product,project_code,project_code_name,promotion_code,promotion_name,loan_date,maturity_date,...,as_output_grade_code,job_type_code,less_than_1_year_balance,less_than_3_year_balance,less_than_5_year_balance,exceeding_5_year_balance,rl_suspention_y/n,customer_total_credit_line,sam_y/n,suing_y/n
0,D,"=""11200001592""","=""1000163020000364""",C4C,V002,01:C4C_068_180,,,2020-02-11,2025-03-01,...,,2,6416.62,0.00,0.0,0.0,N,140000.00,N,N
1,D,"=""11200001210""","=""1000163020001034""",C4C,V003,Re_01:C4C,,,2020-02-28,2025-03-01,...,,7,6676.18,0.00,0.0,0.0,N,290000.00,N,N
2,D,"=""11200000612""","=""1000163030000219""",C4C,V003,Re_01:C4C,,,2020-03-18,2025-04-01,...,,2,6473.42,0.00,0.0,0.0,N,280000.00,N,N
3,D,"=""11200001303""","=""1000163030000260""",C4C,V002,01:C4C_068_180,,,2020-03-25,2025-04-01,...,,9,9256.07,0.00,0.0,0.0,N,400000.00,N,N
4,D,"=""11200001023""","=""1000163030000332""",C4C,V003,Re_01:C4C,,,2020-03-31,2025-04-01,...,,2,27479.92,0.00,0.0,0.0,N,400000.00,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288370,D,"=""13210012277""",9010221122000507,TL,PLD2021070,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,14972.66,40931.85,409.2,0.0,N,205411.79,N,N
1288371,D,"=""13210012644""",9010221122000566,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2026-01-01,...,,,7434.04,274.10,0.0,0.0,N,45667.25,N,N
1288372,D,"=""13210001266""",9010221122000817,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,7546.63,20065.63,0.0,0.0,N,89316.84,N,N
1288373,D,"=""13210005782""",9010221122000931,TL,PLD2020023,ปรับโครงสร้างหนี้_จากRL_22%_48ด,,,2021-12-08,2025-12-01,...,,,16696.03,0.00,0.0,0.0,N,106159.76,N,N


In [83]:
import re
data_tdr['Loan No'] = data_tdr['Loan No'].astype(str).apply(lambda x: re.sub(r'\D', '', x))

In [84]:
data_tdr

,H,Customer No,Loan No,Product,Project Code,Project Code Name,Promotion Code,Promotion Name,Loan Date,Maturity Date,...,AS Output Grade Code,Job Type Code,Less Than 1 Year Balance,Less Than 3 Year Balance,Less Than 5 Year Balance,Exceeding 5 Year Balance,RL Suspention Y/N,Customer Total Credit Line,SAM Y/N,SUING Y/N
0,D,"=""11150019114""",1000158110010231,TL,P012,Activity PL,,,2015-11-12,2017-12-01,...,,6,100.00,0.00,0.0,0.0,N,31500.00,N,N
1,D,"=""11150011505""",1000158120003165,TL,P014,Walk in PL,,,2015-12-11,2018-07-01,...,,5,100.00,0.00,0.0,0.0,N,30600.00,N,N
2,D,"=""11160019055""",1000159040033450,TL,P018,Webpak,,,2016-04-27,2018-05-01,...,,9,90.00,0.00,0.0,0.0,N,32500.00,N,N
3,D,"=""11160011523""",1000159050040751,TL,P014,Walk in PL,,,2016-05-26,2019-06-01,...,,8,1.60,0.00,0.0,0.0,N,38100.00,N,N
4,D,"=""11160048826""",1000159050046961,TL,P018,Webpak,,,2016-05-30,2019-06-01,...,,9,90.00,0.00,0.0,0.0,N,31500.00,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4385,D,"=""13210012277""",9010221122000507,TL,PLD2021070,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,14972.66,40931.85,409.2,0.0,N,205411.79,N,N
4386,D,"=""13210012644""",9010221122000566,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2026-01-01,...,,,7434.04,274.10,0.0,0.0,N,45667.25,N,N
4387,D,"=""13210001266""",9010221122000817,TL,PLD2021035,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3,,,2021-12-01,2028-01-01,...,,,7546.63,20065.63,0.0,0.0,N,89316.84,N,N
4388,D,"=""13210005782""",9010221122000931,TL,PLD2020023,ปรับโครงสร้างหนี้_จากRL_22%_48ด,,,2021-12-08,2025-12-01,...,,,16696.03,0.00,0.0,0.0,N,106159.76,N,N


In [85]:
data_tdr = data_tdr.loc[
    (data_tdr['Product'] == 'TL') & 
    (
        data_tdr['Project Code Name'].str.contains('ปรับโครงสร้าง', na=False) | 
        data_tdr['Project Code Name'].str.contains('TDR', na=False) | 
        data_tdr['Project Code Name'].str.contains('DR', na=False)
    )
].reset_index(drop=True)[['Loan No','Project Code Name']]

In [86]:
data_tdr

,Loan No,Project Code Name
0,7220150100000139,TDR
1,7220150100000141,TDR
2,7220150100000142,TDR
3,7220150100000146,TDR
4,7220150100000148,TDR
...,...,...
3312,9010221122000507,ปรับโครงสร้างหนี้_20.9%_COVID_Phase3
3313,9010221122000566,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3
3314,9010221122000817,ปรับโครงสร้างหนี้_จากRL_22%_COVID_Phase3
3315,9010221122000931,ปรับโครงสร้างหนี้_จากRL_22%_48ด


merge

In [87]:
data_cscore = data_all_digit.merge(data_tdr, left_on='CONTRACT_NO', right_on='Loan No', how='left')

In [88]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'NATIONAL_ID', 'GROSS_INCOME',
       'OCCUPATION_TYPE', 'OCCUPATION_NAME', 'CONTRACT_DATE',
       'CONTRACT_PERIOD', 'AGE', 'PRINCIPAL_BAL',
       ...
       'Status LM', 'BUCKET_SCOREAcc_20250201_0830mnt',
       'BUCKET_SCOREAcc_20250101_0830mnt', 'BUCKET_SCOREAcc_20241201_0830mnt',
       'BUCKET_SCOREAcc_20241101_0830mnt', 'BUCKET_SCOREAcc_20241001_0830mnt',
       'BUCKET_SCOREAcc_20240901_0830mnt', 'AVG_DIGIT1', 'Loan No_y',
       'Project Code Name'],
      dtype='object', length=114)

In [89]:
# Define conditions
conditions = [
    data_cscore['Project Code Name'].notna(),      # A is not null
    data_cscore['MOB'] < 7,          # A is less than 7
    data_cscore['AVG_DIGIT1'] > 3  ,
    data_cscore['AVG_DIGIT1'] > 1,# C is equal to 3
    data_cscore['AVG_DIGIT1'] >= 0,
]

# Define corresponding values
choices = ['H', 'D', 'H','M','L']

data_cscore['TOTAL_DIGIT1']= np.select(conditions, choices, default='Other')

In [90]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'NATIONAL_ID', 'GROSS_INCOME',
       'OCCUPATION_TYPE', 'OCCUPATION_NAME', 'CONTRACT_DATE',
       'CONTRACT_PERIOD', 'AGE', 'PRINCIPAL_BAL',
       ...
       'BUCKET_SCOREAcc_20250201_0830mnt', 'BUCKET_SCOREAcc_20250101_0830mnt',
       'BUCKET_SCOREAcc_20241201_0830mnt', 'BUCKET_SCOREAcc_20241101_0830mnt',
       'BUCKET_SCOREAcc_20241001_0830mnt', 'BUCKET_SCOREAcc_20240901_0830mnt',
       'AVG_DIGIT1', 'Loan No_y', 'Project Code Name', 'TOTAL_DIGIT1'],
      dtype='object', length=115)

In [338]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'GROSS_INCOME', 'OCCUPATION_NAME',
       'CONTRACT_DATE', 'CONTRACT_PERIOD', 'AGE', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'BUCKET_SCORE0mnt',
       'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt', 'BUCKET_SCORE3mnt',
       'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt', 'AVG_DIGIT1', 'TOTAL_DIGIT1',
       'Project Code Name'],
      dtype='object')

In [126]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,BUCKET_SCOREAcc_20250301_0830mnt,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,AVG_DIGIT1,Loan No_y,Project Code Name,TOTAL_DIGIT1
0,2025-03-19 00:00:00.000,8002059101048961,3100400473465,19390.0,8.0,SERVICE JOB,2016-10-13 00:00:00.000,101.0,56.0,86981.96,...,0,0,0,0,0,0,0.000000,NaN,NaN,L
1,2025-03-19 00:00:00.000,5000160091046498,3471200908156,30000.0,10.0,GENERAL LABOR,2017-09-22 00:00:00.000,89.0,53.0,66377.64,...,0,0,0,0,0,0,0.000000,NaN,NaN,L
2,2025-03-19 00:00:00.000,5000460051052371,1849800083805,20500.0,9.0,PRODUCTION LINE LABOR,2017-05-30 00:00:00.000,93.0,33.0,55554.07,...,0,0,0,0,0,0,0.000000,NaN,NaN,L
3,2025-03-19 00:00:00.000,5000361021074351,3361300226376,45180.0,9.0,PRODUCTION LINE LABOR,2018-02-22 00:00:00.000,84.0,45.0,37242.57,...,0,0,0,0,1,0,0.166667,NaN,NaN,L
4,2025-03-19 00:00:00.000,5000160071046565,1330200006075,15500.0,5.0,OFFICE WORK,2017-07-21 00:00:00.000,91.0,40.0,31031.06,...,0,0,0,0,0,0,0.000000,NaN,NaN,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,2025-03-19 00:00:00.000,7220150100000228,1450900019144,15387.0,7.0,DRIVING,2022-01-26 00:00:00.000,37.0,39.0,11881.36,...,6,5,4,3,2,1,3.500000,NaN,NaN,H
36052,2025-03-19 00:00:00.000,9000221102011468,5200100011313,113934.0,5.0,OFFICE WORK,2021-10-27 00:00:00.000,40.0,54.0,223915.63,...,13,6,5,4,3,2,5.500000,NaN,NaN,H
36053,2025-03-19 00:00:00.000,9000220082000911,3560600128873,130317.0,3.0,SENIOR MANAGEMENT,2020-08-13 00:00:00.000,55.0,48.0,82910.89,...,13,6,5,4,3,2,5.500000,NaN,NaN,H
36054,2025-03-19 00:00:00.000,9000220102001658,3100903395243,64750.0,3.0,SENIOR MANAGEMENT,2020-10-19 00:00:00.000,53.0,49.0,42260.78,...,13,6,5,4,3,2,5.500000,NaN,NaN,H


In [91]:
# data_all_digit['score_digit1'] = data_all_digit['TOTAL_DIGIT1'].map({3: 'H', 2: 'M', 1: 'L'})
data_cscore['TOTAL_DIGIT2'] = data_cscore['TOTAL_DIGIT2'].map({3: 'H', 2: 'M', 1: 'L'})

In [92]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,BUCKET_SCOREAcc_20250201_0830mnt,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1,Loan No_y,Project Code Name,TOTAL_DIGIT1
0,2025-03-19 00:00:00.000,8002059101048961,3100400473465,19390.0,8.0,SERVICE JOB,2016-10-13 00:00:00.000,101.0,56.0,86981.96,...,0,0,0,0,0,0,0.000000,NaN,NaN,L
1,2025-03-19 00:00:00.000,5000160091046498,3471200908156,30000.0,10.0,GENERAL LABOR,2017-09-22 00:00:00.000,89.0,53.0,66377.64,...,0,0,0,0,0,NaN,0.000000,NaN,NaN,L
2,2025-03-19 00:00:00.000,5000460051052371,1849800083805,20500.0,9.0,PRODUCTION LINE LABOR,2017-05-30 00:00:00.000,93.0,33.0,55554.07,...,0,0,0,0,0,NaN,0.000000,NaN,NaN,L
3,2025-03-19 00:00:00.000,5000361021074351,3361300226376,45180.0,9.0,PRODUCTION LINE LABOR,2018-02-22 00:00:00.000,84.0,45.0,37242.57,...,0,0,0,1,0,NaN,0.166667,NaN,NaN,L
4,2025-03-19 00:00:00.000,5000160071046565,1330200006075,15500.0,5.0,OFFICE WORK,2017-07-21 00:00:00.000,91.0,40.0,31031.06,...,0,0,0,0,0,NaN,0.000000,NaN,NaN,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,2025-03-19 00:00:00.000,7220150100000228,1450900019144,15387.0,7.0,DRIVING,2022-01-26 00:00:00.000,37.0,39.0,11881.36,...,5,4,3,2,1,NaN,2.500000,NaN,NaN,M
36052,2025-03-19 00:00:00.000,9000221102011468,5200100011313,113934.0,5.0,OFFICE WORK,2021-10-27 00:00:00.000,40.0,54.0,223915.63,...,6,5,4,3,2,2,3.666667,NaN,NaN,H
36053,2025-03-19 00:00:00.000,9000220082000911,3560600128873,130317.0,3.0,SENIOR MANAGEMENT,2020-08-13 00:00:00.000,55.0,48.0,82910.89,...,6,5,4,3,2,1,3.500000,NaN,NaN,H
36054,2025-03-19 00:00:00.000,9000220102001658,3100903395243,64750.0,3.0,SENIOR MANAGEMENT,2020-10-19 00:00:00.000,53.0,49.0,42260.78,...,6,5,4,3,2,3,3.833333,NaN,NaN,H


In [93]:
data_cscore['FINAL_SCORE'] =data_cscore['TOTAL_DIGIT1'].astype(str) + data_cscore['TOTAL_DIGIT2'].astype(str) +'0'+ data_cscore['MOB'].astype(str)

In [94]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1,Loan No_y,Project Code Name,TOTAL_DIGIT1,FINAL_SCORE
0,2025-03-19 00:00:00.000,8002059101048961,3100400473465,19390.0,8.0,SERVICE JOB,2016-10-13 00:00:00.000,101.0,56.0,86981.96,...,0,0,0,0,0,0.000000,NaN,NaN,L,LM07
1,2025-03-19 00:00:00.000,5000160091046498,3471200908156,30000.0,10.0,GENERAL LABOR,2017-09-22 00:00:00.000,89.0,53.0,66377.64,...,0,0,0,0,NaN,0.000000,NaN,NaN,L,LM07
2,2025-03-19 00:00:00.000,5000460051052371,1849800083805,20500.0,9.0,PRODUCTION LINE LABOR,2017-05-30 00:00:00.000,93.0,33.0,55554.07,...,0,0,0,0,NaN,0.000000,NaN,NaN,L,LH07
3,2025-03-19 00:00:00.000,5000361021074351,3361300226376,45180.0,9.0,PRODUCTION LINE LABOR,2018-02-22 00:00:00.000,84.0,45.0,37242.57,...,0,0,1,0,NaN,0.166667,NaN,NaN,L,LH07
4,2025-03-19 00:00:00.000,5000160071046565,1330200006075,15500.0,5.0,OFFICE WORK,2017-07-21 00:00:00.000,91.0,40.0,31031.06,...,0,0,0,0,NaN,0.000000,NaN,NaN,L,LH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,2025-03-19 00:00:00.000,7220150100000228,1450900019144,15387.0,7.0,DRIVING,2022-01-26 00:00:00.000,37.0,39.0,11881.36,...,4,3,2,1,NaN,2.500000,NaN,NaN,M,MH07
36052,2025-03-19 00:00:00.000,9000221102011468,5200100011313,113934.0,5.0,OFFICE WORK,2021-10-27 00:00:00.000,40.0,54.0,223915.63,...,5,4,3,2,2,3.666667,NaN,NaN,H,HL07
36053,2025-03-19 00:00:00.000,9000220082000911,3560600128873,130317.0,3.0,SENIOR MANAGEMENT,2020-08-13 00:00:00.000,55.0,48.0,82910.89,...,5,4,3,2,1,3.500000,NaN,NaN,H,HL07
36054,2025-03-19 00:00:00.000,9000220102001658,3100903395243,64750.0,3.0,SENIOR MANAGEMENT,2020-10-19 00:00:00.000,53.0,49.0,42260.78,...,5,4,3,2,3,3.833333,NaN,NaN,H,HL07


In [95]:
data_cscore.groupby('FINAL_SCORE').size()

FINAL_SCORE
DH06      249
DL06       80
DM06      267
HH06      280
HH07     1169
HL06       32
HL07      172
HM06      131
HM07      515
LH07    21758
LL07     1374
LM07     7039
MH07     2321
ML07      109
MM07      560
dtype: int64

In [97]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,PRINCIPAL_BAL,...,BUCKET_SCOREAcc_20250101_0830mnt,BUCKET_SCOREAcc_20241201_0830mnt,BUCKET_SCOREAcc_20241101_0830mnt,BUCKET_SCOREAcc_20241001_0830mnt,BUCKET_SCOREAcc_20240901_0830mnt,AVG_DIGIT1,Loan No_y,Project Code Name,TOTAL_DIGIT1,FINAL_SCORE
0,2025-03-19 00:00:00.000,8002059101048961,3100400473465,19390.0,8.0,SERVICE JOB,2016-10-13 00:00:00.000,101.0,56.0,86981.96,...,0,0,0,0,0,0.000000,NaN,NaN,L,LM07
1,2025-03-19 00:00:00.000,5000160091046498,3471200908156,30000.0,10.0,GENERAL LABOR,2017-09-22 00:00:00.000,89.0,53.0,66377.64,...,0,0,0,0,NaN,0.000000,NaN,NaN,L,LM07
2,2025-03-19 00:00:00.000,5000460051052371,1849800083805,20500.0,9.0,PRODUCTION LINE LABOR,2017-05-30 00:00:00.000,93.0,33.0,55554.07,...,0,0,0,0,NaN,0.000000,NaN,NaN,L,LH07
3,2025-03-19 00:00:00.000,5000361021074351,3361300226376,45180.0,9.0,PRODUCTION LINE LABOR,2018-02-22 00:00:00.000,84.0,45.0,37242.57,...,0,0,1,0,NaN,0.166667,NaN,NaN,L,LH07
4,2025-03-19 00:00:00.000,5000160071046565,1330200006075,15500.0,5.0,OFFICE WORK,2017-07-21 00:00:00.000,91.0,40.0,31031.06,...,0,0,0,0,NaN,0.000000,NaN,NaN,L,LH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36051,2025-03-19 00:00:00.000,7220150100000228,1450900019144,15387.0,7.0,DRIVING,2022-01-26 00:00:00.000,37.0,39.0,11881.36,...,4,3,2,1,NaN,2.500000,NaN,NaN,M,MH07
36052,2025-03-19 00:00:00.000,9000221102011468,5200100011313,113934.0,5.0,OFFICE WORK,2021-10-27 00:00:00.000,40.0,54.0,223915.63,...,5,4,3,2,2,3.666667,NaN,NaN,H,HL07
36053,2025-03-19 00:00:00.000,9000220082000911,3560600128873,130317.0,3.0,SENIOR MANAGEMENT,2020-08-13 00:00:00.000,55.0,48.0,82910.89,...,5,4,3,2,1,3.500000,NaN,NaN,H,HL07
36054,2025-03-19 00:00:00.000,9000220102001658,3100903395243,64750.0,3.0,SENIOR MANAGEMENT,2020-10-19 00:00:00.000,53.0,49.0,42260.78,...,5,4,3,2,3,3.833333,NaN,NaN,H,HL07


In [96]:
data_cscore.to_csv('./output/cscoreoutput.csv')

=====ย้อนหลัง====

In [345]:
data_cscore.columns

Index(['AS_OF_DATE', 'CONTRACT_NO', 'GROSS_INCOME', 'OCCUPATION_NAME',
       'CONTRACT_DATE', 'CONTRACT_PERIOD', 'AGE', 'MOB', 'DIGIT2INCOME',
       'DIGIT2AGE', 'DIGIT2JOB', 'TOTAL_DIGIT2', 'BUCKET_SCORE0mnt',
       'BUCKET_SCORE1mnt', 'BUCKET_SCORE2mnt', 'BUCKET_SCORE3mnt',
       'BUCKET_SCORE4mnt', 'BUCKET_SCORE5mnt', 'AVG_DIGIT1', 'TOTAL_DIGIT1',
       'Project Code Name', 'FINAL_SCORE'],
      dtype='object')

==

mapassign

In [44]:
file = glob.glob("./input/assisgn_data/*.xlsx") 
data_assign = pd.read_excel(file[0])

In [45]:
file[0].split('\\')[1]

"3.Total assign Mar'25 (1).xlsx"

In [48]:
data_assign.columns


Index(['Base date', 'Customer No', 'Customer ID No', 'Principal balance',
       'Total Unclaimed Amt.', 'Repayment Target Amount',
       'Overdue days(Morning)', 'Overdue Cnt(Morning)', 'Overdue days',
       'Num of Overdue months', 'Overdue Months Division Code', 'Corp. Info.',
       'Sales product code', 'Loan No.', 'Customer Name', 'Customer ID No.',
       'Mobile No.', 'Address(Home)', 'Post code', 'Phone No.(Home)',
       'Address(Office)', 'Post code.1', 'Biz phone No.',
       'Address (place of residence)', 'Post Code', 'Phone No.',
       'Delivery Address', 'Post code.2', 'Delivery phone No.',
       'Customer relationship', 'Related person', 'Phone No. 1',
       'Customer relationship.1', 'Related person.1', 'Phone No. 2',
       'Branch code', 'Outstanding Balance', 'Loan date', 'Installment Count',
       'Product Name', 'Last Money Received date', 'Last payment Amt.',
       'Loan Amt.', 'Principal', 'Monthly payment Amt.',
       ' First_Settlement_date', 'Last pa

In [53]:
data_assign = data_assign[['Loan No.','Old OA','New OA','Overdue Months Division Code']]
data_assign

,Loan No.,Old OA,New OA,Overdue Months Division Code,Overdue Months Division Code
0,7220730100003250,KOR,KOR,Grace_X-Day,Grace_X-Day
1,7221030100003286,NaN,Delay calling,Grace_X-Day,Grace_X-Day
2,7230340100241933,Delay calling,Delay calling,Grace_X-Day,Grace_X-Day
3,7230530100570947,PCN,KOR,OD2,OD2
4,7230940101310936,NaN,Delay calling,Grace_X-Day,Grace_X-Day
...,...,...,...,...,...
40416,7240450102627362,KWC,KWC,OD6,OD6
40417,7240550102879930,GMV,MMN,OD6,OD6
40418,7240750103341461,KWC,KWC,OD6,OD6
40419,7240650103061703,MMN,KWC,OD6,OD6


In [54]:
data_assign['Loan No.'] = data_assign['Loan No.'].astype(str)

In [55]:
data_assign

,Loan No.,Old OA,New OA,Overdue Months Division Code,Overdue Months Division Code
0,7220730100003250,KOR,KOR,Grace_X-Day,Grace_X-Day
1,7221030100003286,NaN,Delay calling,Grace_X-Day,Grace_X-Day
2,7230340100241933,Delay calling,Delay calling,Grace_X-Day,Grace_X-Day
3,7230530100570947,PCN,KOR,OD2,OD2
4,7230940101310936,NaN,Delay calling,Grace_X-Day,Grace_X-Day
...,...,...,...,...,...
40416,7240450102627362,KWC,KWC,OD6,OD6
40417,7240550102879930,GMV,MMN,OD6,OD6
40418,7240750103341461,KWC,KWC,OD6,OD6
40419,7240650103061703,MMN,KWC,OD6,OD6


In [56]:
data_cscore

,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,FLOORED_VALUE,OCCUPATION_TYPE,OCCUPATION_NAME,CONTRACT_DATE,CONTRACT_PERIOD,AGE,...,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,Loan No_y,Project Code Name,TOTAL_DIGIT1,FINAL_SCORE
0,2025-03-09 00:00:00.000,7230950101266462,1709700073957,30000.0,30000.0,5.0,OFFICE WORK,2023-09-21 00:00:00.000,17.0,36.0,...,1,0,5,5,4,2.833333,7230950101266462,ปรับโครงสร้างหนี้_DR_Collection_15%,H,HH07
1,2025-03-09 00:00:00.000,5000161041003419,3300800674709,15611.0,15000.0,9.0,PRODUCTION LINE LABOR,2018-04-02 00:00:00.000,83.0,47.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,OtherH07
2,2025-03-09 00:00:00.000,7230410100429114,1401500153197,12224.0,10000.0,9.0,PRODUCTION LINE LABOR,2023-04-25 00:00:00.000,22.0,30.0,...,13,13,9,8,7,10.500000,NaN,NaN,H,HH07
3,2025-03-09 00:00:00.000,5000461011069133,1102001592652,10000.0,10000.0,6.0,SALES,2018-01-29 00:00:00.000,85.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,OtherH07
4,2025-03-09 00:00:00.000,5000159121088434,1100200920712,10250.0,10000.0,7.0,DRIVING,2016-12-28 00:00:00.000,98.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Other,OtherH07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323431,2025-03-09 00:00:00.000,7250270104833429,1910100097898,45000.0,45000.0,5.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DM06
5323432,2025-03-09 00:00:00.000,7250270104828777,1600100045670,17500.0,15000.0,5.0,OFFICE WORK,2025-02-23 00:00:00.000,0.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,DH06
5323433,2025-03-09 00:00:00.000,7250270104748231,3360101417316,25000.0,25000.0,10.0,GENERAL LABOR,2025-02-09 00:00:00.000,1.0,52.0,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,D,DM06
5323434,2025-03-09 00:00:00.000,7250170104643668,3800400133170,35000.0,35000.0,1.0,REGISTERED BUSINESS OWNER,2025-01-24 00:00:00.000,1.0,42.0,...,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,D,DH06


In [57]:
cscore_assigned = data_assign.merge(data_cscore,left_on='Loan No.',right_on='CONTRACT_NO',how='left')

In [58]:
cscore_assigned.columns

Index(['Loan No.', 'Old OA', 'New OA', 'Overdue Months Division Code',
       'Overdue Months Division Code', 'AS_OF_DATE', 'CONTRACT_NO',
       'NATIONAL_ID', 'GROSS_INCOME', 'FLOORED_VALUE', 'OCCUPATION_TYPE',
       'OCCUPATION_NAME', 'CONTRACT_DATE', 'CONTRACT_PERIOD', 'AGE',
       'PRINCIPAL_BAL', 'MOB', 'DIGIT2INCOME', 'DIGIT2AGE', 'DIGIT2JOB',
       'TOTAL_DIGIT2', 'Loan No_x', 'BUCKET_SCORE0mnt', 'BUCKET_SCORE1mnt',
       'BUCKET_SCORE2mnt', 'BUCKET_SCORE3mnt', 'BUCKET_SCORE4mnt',
       'BUCKET_SCORE5mnt', 'AVG_DIGIT1', 'Loan No_y', 'Project Code Name',
       'TOTAL_DIGIT1', 'FINAL_SCORE'],
      dtype='object')

In [59]:
cscore_assigned

,Loan No.,Old OA,New OA,Overdue Months Division Code,Overdue Months Division Code,AS_OF_DATE,CONTRACT_NO,NATIONAL_ID,GROSS_INCOME,FLOORED_VALUE,...,BUCKET_SCORE1mnt,BUCKET_SCORE2mnt,BUCKET_SCORE3mnt,BUCKET_SCORE4mnt,BUCKET_SCORE5mnt,AVG_DIGIT1,Loan No_y,Project Code Name,TOTAL_DIGIT1,FINAL_SCORE
0,7220730100003250,KOR,KOR,Grace_X-Day,Grace_X-Day,2025-03-09 00:00:00.000,7220730100003250,3240900116555,12900.0,10000.0,...,0,0,0,0,0,0.000000,NaN,NaN,L,LH07
1,7221030100003286,NaN,Delay calling,Grace_X-Day,Grace_X-Day,2025-03-09 00:00:00.000,7221030100003286,3100700619422,42000.0,40000.0,...,0,0,0,0,0,0.000000,NaN,NaN,L,LM07
2,7230340100241933,Delay calling,Delay calling,Grace_X-Day,Grace_X-Day,2025-03-09 00:00:00.000,7230340100241933,3101300732206,17125.0,15000.0,...,0,0,0,0,0,0.000000,NaN,NaN,L,LH07
3,7230530100570947,PCN,KOR,OD2,OD2,2025-03-09 00:00:00.000,7230530100570947,3321200109131,23290.0,20000.0,...,1,0,0,0,0,0.500000,NaN,NaN,L,LM07
4,7230940101310936,NaN,Delay calling,Grace_X-Day,Grace_X-Day,2025-03-09 00:00:00.000,7230940101310936,3100503160974,45000.0,45000.0,...,0,0,0,0,0,0.000000,NaN,NaN,L,LM07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40416,7240450102627362,KWC,KWC,OD6,OD6,2025-03-09 00:00:00.000,7240450102627362,3450500230537,11195.0,10000.0,...,5,5,5,4,3,4.666667,7240450102627362,ปรับโครงสร้างหนี้_DR_Collection_15%,H,HH07
40417,7240550102879930,GMV,MMN,OD6,OD6,2025-03-09 00:00:00.000,7240550102879930,1910400006979,11000.0,10000.0,...,5,4,3,2,1,3.500000,7240550102879930,ปรับโครงสร้างหนี้_TDR_Collection_15%,H,HH07
40418,7240750103341461,KWC,KWC,OD6,OD6,2025-03-09 00:00:00.000,7240750103341461,1260300010989,9728.0,5000.0,...,5,4,3,2,1,3.500000,7240750103341461,ปรับโครงสร้างหนี้_PD_Collection_15%,H,HH07
40419,7240650103061703,MMN,KWC,OD6,OD6,2025-03-09 00:00:00.000,7240650103061703,5401300004930,9569.0,5000.0,...,5,4,3,2,1,3.500000,7240650103061703,ปรับโครงสร้างหนี้_DR_Collection_15%,H,HH07


In [60]:
file = glob.glob("./input/assisgn_data/*.xlsx") 
file_path = file[0].split('\\')[1]


In [61]:
cscore_assigned.to_csv(f'./output/1assign_{file_path}.csv')

---

In [69]:
file = glob.glob("./input/acc_current/*.csv") 


In [70]:
file[0].split('\\')[1]

'Acc_20250301_0830.csv'

In [66]:
file = re.sub(r'\.csv$', '', file[0].split('\\')[1])

In [67]:
file

'Acc_20250301_0830'